In [29]:
import xml.etree.ElementTree as ET
import json
import re

def safe_float(text):
    """Convert a string to float after removing commas."""
    if text is None:
        return 0.0
    return float(re.sub(r",", "", text.strip()))

def parse_decision_logs(root, hero):
    """
    Parse the XML tree and, for every opponent decision (action) in rounds 1 and later
    (preflop and beyond), produce a snapshot log capturing the game state at the moment
    BEFORE the action is processed.
    
    In this snapshot:
      - The snapshot does NOT include any actions from round 0 (blinds/antes).
      - The re-indexed action counter is incremented for every action in rounds ≥ 1,
        so that no action gets a number of 0.
      - Only opponent actions are logged (the hero's actions are skipped).
      - The snapshot shows the acting opponent's hole cards (if available); the hero's
        or other players' hole cards are not revealed in the snapshot.
    """
    logs = []

    for game in root.findall('game'):
        general = game.find('general')
        if general is None:
            continue

        # --- Extract blinds and ante ---
        small_blind_elem = general.find('smallblind')
        big_blind_elem = general.find('bigblind')
        ante_elem = general.find('ante')
        small_blind = safe_float(small_blind_elem.text) if small_blind_elem is not None else 0.0
        big_blind = safe_float(big_blind_elem.text) if big_blind_elem is not None else 0.0
        ante = safe_float(ante_elem.text) if ante_elem is not None else 0.0

        # --- Process players ---
        players = {}        # key: player name, value: player info
        active_players = {} # tracks whether a player is still in the hand
        players_elem = general.find('players')
        if players_elem is None:
            continue

        for player_elem in players_elem.findall('player'):
            name = player_elem.attrib.get('name')
            seat = int(player_elem.attrib.get('seat'))
            is_dealer = (player_elem.attrib.get('dealer') == "1")
            chips = safe_float(player_elem.attrib.get('chips'))
            bet = safe_float(player_elem.attrib.get('bet'))
            win = safe_float(player_elem.attrib.get('win'))
            players[name] = {
                'seat': seat,
                'is_dealer': is_dealer,
                'chips': chips,
                'bet': bet,
                'win': win
            }
            active_players[name] = True

        # --- Determine relative positions ---
        # Sort players by seat number.
        sorted_players = sorted(players.items(), key=lambda item: item[1]['seat'])
        # Find the dealer (button). If none is marked, assume the first player is the dealer.
        dealer_index = None
        for i, (name, info) in enumerate(sorted_players):
            if info['is_dealer']:
                dealer_index = i
                break
        if dealer_index is None:
            dealer_index = 0

        button_player = sorted_players[dealer_index][0]
        small_blind_player = sorted_players[(dealer_index + 1) % len(sorted_players)][0]
        big_blind_player = sorted_players[(dealer_index + 2) % len(sorted_players)][0]

        # Assign relative position codes: small blind = 0, big blind = 1, button = 2, others = 3.
        player_positions = {}
        for name in players:
            if name == small_blind_player:
                players[name]['position_relative'] = 0
            elif name == big_blind_player:
                players[name]['position_relative'] = 1
            elif name == button_player:
                players[name]['position_relative'] = 2
            else:
                players[name]['position_relative'] = 3
            player_positions[name] = players[name]['position_relative']

        # Skip this game if the hero is not present.
        if hero not in players:
            continue

        # --- Initialize state variables ---
        pot_size = 0.0
        cumulative_actions = []  # will contain actions from rounds >= 1 only
        board_cards = []         # community cards seen so far
        pocket_cards = {}        # updated when encountering <cards type="Pocket">
        snapshot_action_counter = 0  # counter for actions in rounds >= 1

        # --- Process rounds in order (sorted by round number) ---
        rounds = game.findall('round')
        rounds = sorted(rounds, key=lambda r: int(r.attrib.get('no')))
        for r in rounds:
            round_no = int(r.attrib.get('no'))
            for elem in r:
                if elem.tag == 'cards':
                    card_type = elem.attrib.get('type')
                    if card_type == 'Pocket':
                        # Record pocket cards for the given player.
                        player = elem.attrib.get('player')
                        text = elem.text.strip() if elem.text else ""
                        cards = text.split()
                        # If cards are hidden (e.g., "X X"), record as unknown.
                        if any(card.upper().startswith("X") for card in cards):
                            pocket_cards[player] = ["unknown", "unknown"]
                        else:
                            pocket_cards[player] = cards
                    else:
                        # Community cards (flop, turn, river)
                        cards = elem.text.split() if elem.text else []
                        board_cards.extend(cards)
                elif elem.tag == 'action':
                    # Build an action dictionary.
                    action_details = {
                        'round': round_no,
                        'player': elem.attrib.get('player'),
                        'action_type': int(elem.attrib.get('type')),
                        'action_sum': safe_float(elem.attrib.get('sum'))
                    }

                    # For actions in round 0 (blinds/antes), update state only (do not record them).
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        # (If desired, you could update pocket_cards from round 0 as well.)
                        continue

                    # For actions in rounds >= 1, first assign a new re-indexed action number.
                    snapshot_action_counter += 1
                    # Copy the action_details and add the new action number.
                    action_details['action_no'] = snapshot_action_counter

                    # If the acting player is not the hero, produce a snapshot BEFORE processing the action.
                    # (The snapshot’s "previous_actions" will be the cumulative_actions so far.)
                    if action_details['player'] != hero:
                        is_button = (action_details['player'] == button_player)
                        actor_cards = pocket_cards.get(action_details['player'], ["unknown", "unknown"])
                        # Build the snapshot.
                        snapshot = {
                            "gamecode": game.attrib.get("gamecode"),
                            "round_no": round_no,
                            "current_street": (
                                "preflop" if round_no == 1 else
                                "flop" if round_no == 2 else
                                "turn" if round_no == 3 else
                                "river" if round_no == 4 else "unknown"
                            ),
                            "blinds": {
                                "small_blind": small_blind,
                                "big_blind": big_blind,
                                "ante": ante
                            },
                            "player_positions": player_positions,
                            "player_stacks": {name: players[name]['chips'] for name in players},
                            "pot_size": pot_size,  # state BEFORE processing this action
                            "board_cards": board_cards.copy(),
                            "previous_actions": cumulative_actions.copy(),  # only round>=1 actions so far
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for active in active_players.values() if active),
                            "is_button": is_button,
                            "actor_hole_cards": pocket_cards.get(action_details['player'], ["unknown", "unknown"])
                        }
                        logs.append(snapshot)

                    # Now update the state: add this action to the cumulative history.
                    cumulative_actions.append(action_details)
                    pot_size += action_details['action_sum']
                    # Mark the actor as inactive if the action is a fold (we assume type==0 means fold).
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False

    return logs

if __name__ == '__main__':
    try:
        # Parse the XML file (adjust 'test.xml' to your filename)
        tree = ET.parse('test.xml')
        root = tree.getroot()

        # Determine the hero's nickname from the session-level <nickname> element.
        session_general = root.find('general')
        if session_general is not None and session_general.find('nickname') is not None:
            hero = session_general.find('nickname').text.strip()
        else:
            hero = "mmpq8j6x8"  # fallback if not found

        # Parse opponent decision snapshots (skipping hero actions and round 0 actions)
        logs = parse_decision_logs(root, hero)
        print(json.dumps(logs, indent=4))
    except ET.ParseError as e:
        print("Error parsing XML file:", e)


[
    {
        "gamecode": "10136070425",
        "round_no": 1,
        "current_street": "preflop",
        "blinds": {
            "small_blind": 10.0,
            "big_blind": 20.0,
            "ante": 0.0
        },
        "player_positions": {
            "GGM3M": 0,
            "Bekocabron": 1,
            "mmpq8j6x8": 2
        },
        "player_stacks": {
            "GGM3M": 500.0,
            "Bekocabron": 500.0,
            "mmpq8j6x8": 500.0
        },
        "pot_size": 30.0,
        "board_cards": [],
        "previous_actions": [
            {
                "round": 1,
                "player": "mmpq8j6x8",
                "action_type": 0,
                "action_sum": 0.0,
                "action_no": 1
            }
        ],
        "action": {
            "round": 1,
            "player": "GGM3M",
            "action_type": 3,
            "action_sum": 10.0,
            "action_no": 2
        },
        "players_remaining": 2,
        "is_button": false,
   